First, we need to know how far back to go. We want equal number of days before and after November 18, 2014.

In [4]:
from datetime import datetime, timedelta
pivot = datetime.strptime('11/18/2014', '%m/%d/%Y')
today = datetime.now()
print today - pivot

434 days, 17:15:25.604433


In [5]:
period = timedelta(days=434)
print pivot - period

2013-09-10 00:00:00


Since this was run on 1/26/2016, 434 days after November 18, 2014, we want to go to 434 days before that date, which is 9/10/2013.